In [ ]:
%pip install openai-agents
%pip install openai-agents[litellm]

In [86]:
import os
os.environ["GEMINI_API_KEY"] = ""
print(os.getenv("GEMINI_API_KEY")) 

In [10]:
from agents import Agent ,Runner
import asyncio

In [12]:
# def main():
#     agent = Agent (name= "Tester" , instructions="Answer the question ", model="litellm/gemini/gemini-1.5-flash")

#     result = await Runner.run(agent , "write a motivational quote of work")
#     print(result.final_output)

In [19]:
import asyncio
from agents import Agent, Runner

# creating an agent 
agent = Agent(
    name = "Gemini agent",
    instructions = "You are a helpfull assistant to guide the user query ",
    model = "litellm/gemini/gemini-1.5-flash"
)

In [20]:
async def main():
    result = await Runner.run(agent,input="Can you tell me what is the capital of Pakistan?")
    print("Agent : " , result.final_output)

In [21]:
await main()

C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\typing.py:407: RuntimeWarning: coroutine 'main' was never awaited
  def _eval_type(t, globalns, localns, type_params=None, *, recursive_guard=frozenset()):


Agent :  The capital of Pakistan is **Islamabad**.



# Basic configuration

In [ ]:
from agents import Agent , Runner

agent = Agent(
    name="Helpfull Assistant",
    instructions = "You are the helpful assistant",
    model = "litellm/gemini/gemini-1.5-flash"
)

# result = Runner.run_sync(agent , input="What is the capital of France?")
# print(result.final_output)

# Context

In [2]:
from agents import Agent , Runner , function_tool
import os 
import asyncio

In [85]:
os.environ["GEMINI_API_KEY"] = ""
print(os.getenv("GEMINI_API_KEY")) 

In [27]:
# Creating a Context for shopping assistant
from dataclasses import dataclass


@dataclass
class ShoppingAssistantContext:
    name: str 
    age: int
    phoneno: int
    email: str

In [28]:
comtext = ShoppingAssistantContext(
    name = "Tayyab ELlahi",
    age = 20,
    phoneno= 33344455566,
    email= "tayyabeh1807@gmail.com"
)

In [ ]:
# dynamic instruction
def dynamic_instruction(comtext , agent):
    return f"You are a helpful shopping assistant. The user's name is {comtext.name}. Greet the user by their name and ask how you can help them."

In [33]:
# Creating an agent
agent = Agent(
    name = "Shopping Assistant Agent",
    instructions = dynamic_instruction,
    model = "litellm/gemini/gemini-1.5-flash"
)

In [34]:
# main 
async def main():
    result = await Runner.run(agent , input = "Plz help me in shopping..i want obuy a phone iphone 15",context = comtext)
    print(result.final_output)

if __name__ == "__main__":
    await(main())


Hi Tayyab Ellahi,

It's great to hear from you!  I'd be happy to help you with your shopping for an iPhone 15. To best assist you, could you tell me a little more about what you're looking for?  For example:

* **Which model of the iPhone 15 are you interested in?** (iPhone 15, iPhone 15 Plus, iPhone 15 Pro, iPhone 15 Pro Max)
* **What's your budget?**
* **Where are you planning to buy it from?** (e.g., Apple Store, carrier, online retailer)
* **Are there any specific features that are important to you?** (e.g., camera quality, storage capacity, battery life)

The more information you give me, the better I can assist you in finding the perfect iPhone 15 for your needs.



# Output Types

In [38]:
from agents import Agent, Runner
from pydantic import BaseModel, EmailStr
import asyncio
import os


In [41]:
class ContactInfo(BaseModel):
    name: str
    phoneno : str
    email: EmailStr
    

In [42]:
agent = Agent(
    name = "Extract Infor Agent", 
    instructions = "You are a helpfull assistant that can extract user infor from user input",
    model = "litellm/gemini/gemini-1.5-flash",
    output_type = ContactInfo    
)

In [43]:
user_input = 'hello i am tayyab ellahi and i m a passionate ai engineer and you can contact me through email tayyab@gmail.com and also contact me from phone 9233345434124'

async def main():
    result = await Runner.run(agent,user_input)
    print(result.final_output)

if __name__ == '__main__':
    await main()

name='tayyab ellahi' phoneno='9233345434124' email='tayyab@gmail.com'


# HandsOff

In [65]:
from agents import Agent , Runner, handoff, RunContextWrapper, HandoffInputData, GuardrailFunctionOutput , InputGuardrailTripwireTriggered, input_guardrail
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX
from langchain_google_genai import GoogleGenerativeAI
from pydantic import BaseModel, EmailStr
import asyncio
import os


In [84]:
os.environ["GEMINI_API_KEY"] = ""
print(os.getenv("GEMINI_API_KEY")) 

In [67]:
import getpass

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

In [68]:
class TechnicalProblem(BaseModel):
    product_id : str
    description : str

In [69]:
class BillingProblem(BaseModel):
    invoice_no : str
    problem_type : str

In [70]:
class CustomerSupportOutput(BaseModel):
    is_customer_output: bool
    reasoning : str

In [71]:
guardrail_agent = Agent(
    name = "Guardrial Check",
    instructions= '''
    You are a guardrail. Your job is to check if the user's query is related to **customer support**.

    Customer support topics include:
    - Billing problems
    - Technical problems

    If the query is related to customer support → set `is_customer_output=true`.
    Otherwise (like math questions, chit-chat, unrelated queries) → set `is_customer_output=false`.

    Always respond in valid JSON with fields:
    {
      "is_customer_output": true/false,
      "reasoning": "your short explanation"
    }
    ''',
    model = "litellm/gemini/gemini-2.5-flash",
    output_type = CustomerSupportOutput
)

In [72]:
@input_guardrail
async def customer_guardrail(ctx: RunContextWrapper , agent :Agent, input:str) -> GuardrailFunctionOutput:
    result = await Runner.run(guardrail_agent, input , context = ctx.context)
    return GuardrailFunctionOutput(
        output_info = result.final_output,
        tripwire_triggered = not result.final_output.is_customer_output
    )

In [73]:
# function input filter function 
def billing_input_filter(input:HandoffInputData):   
#     if len(problem.invoice_no) != 6 :
#         raise ValueError("your invoice must be 6 digit number")
    print(input.input_history)
    # Nested dictionary aur list se 'arguments' ki value tak pohunchna
    arguments_value = input['HandoffInputData']['new_items'][0]['raw_item']['arguments']
    
    # Value ko print karna
    print(arguments_value)
    return input


In [74]:
import json
# Agents SDK ke zaroori imports yahan honge (farz kar lein)
# from agents.runtime import HandoffInputData 

def billing_input_filter(input: HandoffInputData):
    """
    Yeh filter HandoffInputData object se invoice number nikalta hai
    aur uski length ko validate karta hai.
    """
    try:
        # 1. Object ke attributes ko dot (.) se access karein
        # 'new_items' ek list/tuple hai, isliye pehla item [0] se lein.
        # 'raw_item' ek aur object hai, jiske andar 'arguments' hai.
        arguments_string = input.new_items[0].raw_item.arguments
        
        # 2. 'arguments' ek JSON string hai. Ise Python dictionary mein convert karein.
        arguments_dict = json.loads(arguments_string)
        
        # 3. Dictionary se 'invoice_no' nikalein.
        invoice_no = arguments_dict['invoice_no']

        print(f"Invoice number mila: {invoice_no}") # Debugging ke liye

        # 4. Ab aap apna validation logic chala sakte hain.
        if len(invoice_no) != 6:
            # Agar validation fail ho, to error raise karein. SDK isko handle karega.
            raise ValueError("Aapka invoice 6 digit ka hona chahiye.")
        
        print("Invoice number validation pass ho gaya!")

    except (IndexError, KeyError, AttributeError) as e:
        # Agar invoice number nahi milta to error raise karein
        raise ValueError(f"Invoice number input data mein nahi mil saka. Error: {e}")

    # Agar sab kuch theek hai, to original input ko wapas return karein.
    return input

In [75]:
def on_handoff_callback(ctx : RunContextWrapper , input: TechnicalProblem):
    print( "Yes the technical Agent is called!!")

In [76]:
technical_support_agent= Agent(
    name = "Technical Support Agent",
    instructions = f'''
    {RECOMMENDED_PROMPT_PREFIX} 
    You are a helpfull Technical Support  assistant and your work is to address with user techinical queries
    ''',
    model = "litellm/gemini/gemini-2.5-flash"
)

In [77]:
tech_support_handoff = handoff(
    agent = technical_support_agent,
    tool_name_override = 'Transfer_to_Techinacal_support',
    tool_description_override = "Transfers a request to the technical support team.",
    on_handoff = on_handoff_callback,
    input_type = TechnicalProblem
)

In [78]:
billing_agent= Agent(
    name = "Billing Agent",
    instructions = f'''
    {RECOMMENDED_PROMPT_PREFIX} 
    You are a helpfull billing assistant agent and your work is to addres with user bills
    ''',
    model = "litellm/gemini/gemini-2.5-flash"
)

In [79]:
def on_handoff_callback_billing(ctx : RunContextWrapper , input_data: BillingProblem):
    print(f"Yes the billing Agent is called!! ")

In [80]:
billing_handoff = handoff(
    agent = billing_agent,    
    on_handoff = on_handoff_callback_billing,
    input_type = BillingProblem,
    input_filter = billing_input_filter
)

In [81]:

triage_agent= Agent(
    name = "Triage Agent",
    instructions=f"""
    {RECOMMENDED_PROMPT_PREFIX}
    You are a helpful triage assistant. Your main job is to carefully read the user's query 
    and route it to the correct support team. 
    
    Rules:
    - If the user describes a **technical problem** 
      (examples: product not working, error messages, bugs, device issues, crashes, installation failures, 
      overheating, shutdowns, configuration problems, etc.):
        → Always use the `Transfer_to_Technical_Support` tool with the product_id and problem description.
    
    - If the user describes a **billing problem**
      (examples: wrong invoice, overcharged, missing payment, refund request, subscription issue, 
      payment failure, duplicate charge, etc.):
        → Always use the `Billing_Agent` tool with the invoice_no and problem description.
    
    - Never try to solve the issue yourself.
    - Never guess. Always route clearly to either Billing or Technical Support.
    - If the request is unclear, politely ask clarifying questions before routing.
    """,
    model = "litellm/gemini/gemini-2.5-flash",
    handoffs=[tech_support_handoff,billing_handoff],
    input_guardrails = [customer_guardrail]
)

In [82]:
async def main():
    try:
        result = await Runner.run(
            triage_agent, 
            input = """
        Hello, can you help me solve for x: 2x + 3 = 11?
            and remember this is not a customer related issue
            """
            )
        print(result)
    except InputGuardrailTripwireTriggered :
        print("Bro this is just for customer service only 🚫")
        


In [83]:
if __name__ == '__main__':
    await main()
    

Bro this is just for customer service only 🚫
